# Building a Module

A module in Python is conceptually not much more than a file with python code in it that you can access and run by importing the module.

When you `import` a module, you are reading the file and making the code contained in it accessible through its alias.

Start by creating a new file called `hello.py` and add the following message in there:

```python
message = 'Hello Jim'
```

and then try executing this code:

```python
import hello as h
h.message
```

Great!

Now, let's change the message. Edit the file to now say:

```python
message = 'Hello Jane'
```

and rexecute the code:

```python
import hello as h
h.message
```

What went wrong?

Since the contents of modules dont usually change very much (except during module development) Python is smart about importing modules and once a module is imported, it doesnt bother re-importing the module. Normally, this is what you want, but in our case when we are developing the module we want to force the module to geet automatically reloaded everytime it changes.

Fortunately, there is a _magic_ command sequence that does exactly that. First, we have to load an extension called `autoreload` by executing the _magic command_ `%load_ext autoreload`. Once you have loaded that extension, you now have access to a new _magic command_ called `%autoreload` which supports different modes of autoreload. The mode we want for now is to automatically reload anything that changes, which is mode 2. So, you need to execute:

```python
%load_ext autoreload
%autoreload 2
```

Now try executing:

```python
h.message
```

and you will see that future edits to the hello file will immediately be reloaded.

Try editing the `hello.py` file to:

```python
message = 'Hello John and Jane'
```

and then execute

```python
h.message
```

Now, let's create our new module, which we'll build on through the course, and put the `drawdown` function we created in that module. Create a file called `edhec_risk_kit.py` and copy the follwing code into it.

```python

import pandas as pd

def drawdown(return_series: pd.Series):
    """Takes a time series of asset returns.
       returns a DataFrame with columns for
       the wealth index, 
       the previous peaks, and 
       the percentage drawdown
    """
    wealth_index = 1000*(1+return_series).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks
    return pd.DataFrame({"Wealth": wealth_index, 
                         "Previous Peak": previous_peaks, 
                         "Drawdown": drawdowns})

def get_ffme_returns():
    """
    Load the Fama-French Dataset for the returns of the Top and Bottom Deciles by MarketCap
    """
    me_m = pd.read_csv("data/Portfolios_Formed_on_ME_monthly_EW.csv",
                       header=0, index_col=0, na_values=-99.99)
    rets = me_m[['Lo 10', 'Hi 10']]
    rets.columns = ['SmallCap', 'LargeCap']
    rets = rets/100
    rets.index = pd.to_datetime(rets.index, format="%Y%m").to_period('M')
    return rets

```


In [2]:
import pandas as pd

def drawdown(return_series: pd.Series):
    """Takes a time series of asset returns.
       returns a DataFrame with columns for
       the wealth index, 
       the previous peaks, and 
       the percentage drawdown
    """
    wealth_index = 1000*(1+return_series).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks
    return pd.DataFrame({"Wealth": wealth_index, 
                         "Previous Peak": previous_peaks, 
                         "Drawdown": drawdowns})

def get_ffme_returns():
    """
    Load the Fama-French Dataset for the returns of the Top and Bottom Deciles by MarketCap
    """
    me_m = pd.read_csv("data/Portfolios_Formed_on_ME_monthly_EW.csv",
                       header=0, index_col=0, na_values=-99.99)
    rets = me_m[['Lo 10', 'Hi 10']]
    rets.columns = ['SmallCap', 'LargeCap']
    rets = rets/100
    rets.index = pd.to_datetime(rets.index, format="%Y%m").to_period('M')
    return rets

In [5]:
rets=get_ffme_returns()
drawdown(rets['SmallCap'])

,Wealth,Previous Peak,Drawdown
1926-07,9.855000e+02,9.855000e+02,0.000000
1926-08,1.035958e+03,1.035958e+03,0.000000
1926-09,1.045592e+03,1.045592e+03,0.000000
1926-10,9.949854e+02,1.045592e+03,-0.048400
1926-11,9.872245e+02,1.045592e+03,-0.055822
...,...,...,...
2018-08,2.298938e+09,2.298938e+09,0.000000
2018-09,2.260316e+09,2.298938e+09,-0.016800
2018-10,2.033832e+09,2.298938e+09,-0.115317
2018-11,1.959597e+09,2.298938e+09,-0.147608


In [6]:
rets=get_ffme_returns()
drawdown(rets['LargeCap'])

,Wealth,Previous Peak,Drawdown
1926-07,1.032900e+03,1.032900e+03,0.000000
1926-08,1.071117e+03,1.071117e+03,0.000000
1926-09,1.078294e+03,1.078294e+03,0.000000
1926-10,1.052091e+03,1.078294e+03,-0.024300
1926-11,1.080498e+03,1.080498e+03,0.000000
...,...,...,...
2018-08,4.175915e+06,4.175915e+06,0.000000
2018-09,4.212246e+06,4.212246e+06,0.000000
2018-10,3.935501e+06,4.212246e+06,-0.065700
2018-11,4.035069e+06,4.212246e+06,-0.042062
